<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Общий-вывод:" data-toc-modified-id="Общий-вывод:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод:</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

**Цель исследования**

- защитить данные клиентов страховой компании «Хоть потоп»: разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию;
- защитить данные так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. При этом подбирать наилучшую модель не требуется.

**Описание данных:**

для проведения анализа представлен следующий файл:

- */datasets/insurance.csv*.



- **Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.
- **Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

**План работы:**

1. открыть файл с данными и изучить общую информацию;
2. ответить на вопрос "Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?" и обосновать решение;
3. предложить алгоритм преобразования данных для решения задачи и обосновать, почему качество линейной регрессии не поменяется;
4. запрограммировать этот алгоритм, применив матричные операции;
5. проверить, что качество линейной регрессии из sklearn не отличается до и после преобразования (применить метрику R2);
6. написать вывод.   

## Загрузка данных

На этом этапе мы загрузим данные из представленного файла в датафрейм, изучим полученную информацию и проведем проверку на наличие пропусков и дубликатов.

In [1]:
# импортируем нужные библиотеки
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
# загружаем данные из файла в датафрейм и выводим основную информацию о датафрейме с помощью метода info()
data = pd.read_csv('/datasets/insurance.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Как видим, в нашем датафрейме 5 столбцов (пол, возраст, зарплата, члены семьи, страховые выплаты) и  5 000 строк. Данные в столбцах 3-х столбцах (пол, члены семьи, страховые выплаты) относятся к целочисленному типу, а 2-х (возраст и зарплата) - к вещественному. Переименовывать столбцы с соответствии с требованиями "змеиного регистра" не будем, т.к. это никак не влияет на успешность достижения цели нашего проекта и, кроме того, название столбцов является предельно понятным.

Теперь выведем на экран первые 20 строчек нашего датафрейма, а также посмотрим на распределение по каждому из столбцов.

In [3]:
data.head(20)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [4]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Пол,5000.0,0.4990,0.500049,0.0,0.0,0.0,1.0,1.0
Возраст,5000.0,30.9528,8.440807,18.0,24.0,30.0,37.0,65.0
Зарплата,5000.0,39916.3600,9900.083569,5300.0,33300.0,40200.0,46600.0,79000.0
Члены семьи,5000.0,1.1942,1.091387,0.0,0.0,1.0,2.0,6.0
Страховые выплаты,5000.0,0.1480,0.463183,0.0,0.0,0.0,0.0,5.0


Видим, что:
- в столбце "пол" имеется два типа значений: 0 и 1 (т.е. мужской и женский; при этом из описания проекта не ясно какой именно цифрой закодирован конкретный пол, но для нашего исследования это и не является важным). Среднее значение в столбце меньше 0,5 и медиана - 0, что говорит большем распространении "нулевого пола".
- В датафрейме представлены клиенты в возрасте от 18 до 65 лет (при медиане 30).
- Зарплата принимает значение от 5,3 тыс. руб. до 79 тыс. руб.
- Члены семьи обозначены цифрами (имеется в виду их количество) от 0 до 6.
- Данные в столбце "страховые выпллаты" принимают значение от 0 до 5 (при медиане 0). Это количество страховых выплат клиенту за последние 5 лет (является целевым признаком.

Явных противоречий либо маловероятных значений в представленных выше распределениях не наблюдается. Также видим, что все данные в нашей таблице являются целыми числами, поэтому приведем данные в столбцах "возраст" и "зарплата" также к типу int64.

In [5]:
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')
data.dtypes

Пол                  int64
Возраст              int64
Зарплата             int64
Члены семьи          int64
Страховые выплаты    int64
dtype: object

Теперь все значения выражены целочисленным типом, что в данном случае является более корректным. Теперь проверим на наличие пропусков.

In [6]:
# проверим количество пропусков в % 
pd.DataFrame(round(data.isna().mean()*100,).sort_values(ascending=False)).style.background_gradient('coolwarm')

,0
Пол,0.000000
Возраст,0.000000
Зарплата,0.000000
Члены семьи,0.000000
Страховые выплаты,0.000000


Пропуски отсутствуют. Теперь проверим на наличие дубликатов.

In [7]:
data.duplicated().sum()

153

Видим, что имеется 153 явных дубликата. Осуществлять поиск неявных дубликатов не имеет смысла, т.к. все значения столбцов датафрейма являются числовыми.
При этом необходимо учесть, что в таблице отсутствуют персональные идентификаторы клиента и нельзя исключать вероятность, что одинаковые данные принадлежат разным людям. Кроме того, удаление этих строк никак не влияет на успешность достижения цели нашего проекта. 
Соответственно, полагаю целесообразным не удалять эти выявленные 153 "дубликата".

Таким образом, на данном этапе мы выяснили следующее:

- в нашем датафрейме 5 столбцов (пол, возраст, зарплата, члены семьи, страховые выплаты) и  5 000 строк. Данные в столбцах 3-х столбцах (пол, члены семьи, страховые выплаты) относятся к целочисленному типу, а 2-х (возраст и зарплата) - к вещественному.
- В столбце "пол" имеется два типа значений: 0 и 1 (т.е. мужской и женский; при этом из описания проекта не ясно какой именно цифрой закодирован конкретный пол, но для нашего исследования это и не является важным). Среднее значение в столбце меньше 0,5 и медиана - 0, что говорит большем распространении "нулевого пола".
- В датафрейме представлены клиенты в возрасте от 18 до 65 лет (при медиане 30).
- Зарплата принимает значение от 5,3 тыс. руб. до 79 тыс. руб.
- Члены семьи обозначены цифрами (имеется в виду их количество) от 0 до 6.
- Данные в столбце "страховые выпллаты" принимают значение от 0 до 5 (при медиане 0). Это количество страховых выплат клиенту за последние 5 лет (является целевым признаком.
- Явных противоречий либо маловероятных значений в представленных выше распределениях не наблюдается.
- Все данные в нашей таблице являются целыми числами, поэтому данные в столбцах "возраст" и "зарплата" мы также привели к типу int64.
- Пропуски данных отсутствуют.
- Мы выявили 153 явных дубликата, которые удалять не стали по изложенным выше причинам. Неявные дубликаты отсутствуют.

Теперь данные готовы для проведения исследования и мы можем приступить к следующим шагам нашего проекта.

## Умножение матриц

По сути выполнение нашего проекта сводится к следующему: умножить признаки на любую обратимую матрицу и справнить R2 до и после такого умножения. Кроме того, математически доказать, что R2 не изменится (т.е. не изменятся предсказания). 

Начнем с того, что математически докажем следующее: умножение матрицы признаков  𝑋 на некоторую любую обратимую матрицу 𝑃 не меняет предсказания.
Ниже приведены необходимые нам формулы.

**Предсказания:**

$$
a = Xw
$$

**Задача обучения:**

$$
w = \arg\min_w MSE(Xw, y)
$$

**Формула обучения:**

$$
w = (X^T X)^{-1} X^T y
$$

**Обозначения:**

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

На этом этапе нам необходимо ответить на вопрос "Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?" и обосновать решение: 

- если изменится, привести примеры матриц;
- если не изменится, указать как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Для ответа на вопрос умножим матрицу признаков $X$ на некоторую любую обратимую матрицу $P$.

  
$$
w_P = ((XP)^T XP)^{-1} (XP)^T y
$$   
    
В первой части выражения у нас $((XP)^T XP)^{-1}$. Раскроем скобки:
    
$$
((XP)^T XP)^{-1} = (P^TX^TXP)^{-1} = (P^T)^{-1}(X^TX)^{-1}P^{-1}
$$       
    
 Тогда для $w_P$ получаем:
    
$$
w_P = (P^T)^{-1}(X^TX)^{-1}P^{-1}(XP)^Ty = (P^T)^{-1}(X^TX)^{-1}P^{-1}P^TX^Ty
$$   
      
При этом согласно формуле выше вектор весов линейной регрессии (его изначальное, до кодировки) вычисляется так:
    
$$
w = (X^T X)^{-1} X^T y
$$   
    
Отсюда получаем:
    
$$
w_P = (P^T)^{-1}(X^TX)^{-1}P^{-1}P^TX^Ty = (P^T)^{-1}wP^{-1}P^T
$$   

А, учитывая, что произведение $(P^T)^{-1}P^T$ дает единичную матрицу, получаем итоговое выражение для $w_P$:
    
$$
w_P = wP^{-1}
$$   

Таким образом, ответ на вопрос о том, как связаны параметры линейной регрессии в исходной задаче и в преобразованной, какое соотношение между $w$ и $w_P$, следующий:
    
вектор весов линейной регрессии после умножения матрицы признаков на любую обратимую квадратную матрицу P равен произведению первоначального вектора весов (т.е. до умножения) на матрицу, обратную P.
    
Теперь посмотрим на выражение для предсказаний после кодировки. Оно следующее: 
    
$$
a_p = XPw_p
$$   
    
Подставим вместо $w_P$ выражение $w_P = wP^{-1}$:
    
    
$$
a_p = XPw_p = XPwP^{-1} = Xw
$$      
    
    
Но $Xw = a$ (т.е. предсказания до кодировки). **Поэтому в итоге получаем, что $a = a_p$ (т.е. умножение матрицы признаков $X$ на некоторую любую обратимую квадратную матрицу $P$ не меняет предсказания - новые предсказания равны старым), что и требовалось доказать.**



## Алгоритм преобразования

Как уже отмечалось выше, целью нашего проекта является:
- защитить данные клиентов страховой компании «Хоть потоп»: разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию;
- защитить данные так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. При этом подбирать наилучшую модель не требуется.

**Выберем следующий метод защиты персональной информации: умножим все имеющиеся в датасете признаки на случайную обратимую матрицу**. Как мы уже доказали выше, это не изменит предсказания. В качестве дополнительной проверки измерим R2 модели до и после такого умножения.

На этом шаге мы замерим R2 модели до такого умножения.

In [8]:
# извлекаем признаки
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [9]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print('Метрика R2 модели составляет:', r2_score(target, predictions))

Метрика R2 модели составляет: 0.42494550308169177


## Проверка алгоритма

Теперь умножим все имеющиеся в датасете признаки (features) на случайную обратимую матрицу и снова замерим R2

In [10]:
# сгенерируем случайную матрицу размером 4 на 4 методом np.random.normal()
matr = np.random.normal(size=(4, 4))
display(matr)

array([[-1.65941285,  1.70505192, -1.07001769, -0.08340923],
       [ 2.47794894, -1.17096859, -1.03014491,  0.54553165],
       [-1.26137369,  0.35508591, -0.04928406, -0.06841683],
       [ 2.69712088, -0.8219299 ,  0.70113369, -0.26144704]])

In [11]:
# проверим нашу матрицу на обратимость функцией np.linalg.inv()
np.linalg.inv(matr)

array([[  0.18227822,  -0.11998527,  -1.23623522,   0.01499288],
       [  0.44404853,  -0.84002308,  -4.03012377,  -0.8398207 ],
       [ -0.45277425,  -1.04367788,  -3.720486  ,  -1.05967889],
       [ -0.72980612,  -1.39581769, -10.06075509,  -3.87177917]])

Ошибки нет. Значит, наша матрица обратимая и мы можем продолжать.

In [12]:
# умножаем признаки на нашу матрицу
features_1 = features @ matr

In [13]:
# выведем на экран закодированные признаки
features_1.head(15)

,0,1,2,3
0,-62461.501412,17565.134681,-2487.094310,-3371.452660
1,-47815.517450,13438.578199,-1919.479895,-2575.006404
2,-26416.986972,7422.846078,-1064.839508,-1420.932942
3,-52541.851706,14780.848361,-2075.376169,-2842.048400
4,-32854.130153,9236.660254,-1316.228098,-1770.487698
5,-51606.034659,14508.231965,-2064.610531,-2782.238335
6,-49976.160658,14051.304153,-1996.420670,-2695.478587
7,-48617.946642,13675.459349,-1926.383906,-2628.380415
8,-62600.028526,17606.498116,-2486.871991,-3381.022001
9,-65132.687702,18321.353815,-2581.319536,-3520.037781


Как видим, по получившейся таблице довольно проблематично установить пол, возраст, зарплату и количество членов семьи клиентов страховой компаниий. Таким образом, защитить данные у нас получилось. Теперь замерим метрику R2 на закодированных данных.

In [14]:
model = LinearRegression()
model.fit(features_1, target)
predictions = model.predict(features_1)
print('Метрика R2 модели составляет:', r2_score(target, predictions))

Метрика R2 модели составляет: 0.42494550308169265


Таким образом:

- до кодирования метрика R2 нашей модели составляла: 0.42494550308169177;
- после кодирования метрика R2 составила: 0.42494550308169976.

Т.е. значения R2 до и после кодирования совпадают на 99.9999999999981%. Соответственно, мы можем полагать, что значения метрики не изменилось и цель проекта достигнута.

## Общий вывод:

Целью нашего проекта было защитить данные клиентов страховой компании «Хоть потоп», для чего было необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. При этом защитить данные нужно было так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требовалось.

Для проведения анализа был представлен следующий файл:

- */datasets/insurance.csv*.

**Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.
**Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

На этапе изучения и предобработки даннных мы выяснили следующее:

- в нашем датафрейме 5 столбцов (пол, возраст, зарплата, члены семьи, страховые выплаты) и  5 000 строк. Данные в столбцах 3-х столбцах (пол, члены семьи, страховые выплаты) относятся к целочисленному типу, а 2-х (возраст и зарплата) - к вещественному.
- В столбце "пол" имеется два типа значений: 0 и 1 (т.е. мужской и женский; при этом из описания проекта не ясно какой именно цифрой закодирован конкретный пол, но для нашего исследования это и не является важным). Среднее значение в столбце меньше 0,5 и медиана - 0, что говорит большем распространении "нулевого пола".
- В датафрейме представлены клиенты в возрасте от 18 до 65 лет (при медиане 30).
- Зарплата принимает значение от 5,3 тыс. руб. до 79 тыс. руб.
- Члены семьи обозначены цифрами (имеется в виду их количество) от 0 до 6.
- Данные в столбце "страховые выпллаты" принимают значение от 0 до 5 (при медиане 0). Это количество страховых выплат клиенту за последние 5 лет (является целевым признаком.
- Явных противоречий либо маловероятных значений в представленных выше распределениях не наблюдается.
- Все данные в нашей таблице являются целыми числами, поэтому данные в столбцах "возраст" и "зарплата" мы также привели к типу int64.
- Пропуски данных отсутствуют.
- Выявили 153 явных дубликата, которые удалять не стали по изложенным выше причинам. Неявные дубликаты отсутствуют.

На 2-м шаге проекта мы математически доказали, что умножение матрицы признаков 𝑋 на некоторую любую обратимую матрицу 𝑃 не меняет предсказания (т.е. новые предсказания равны старым). Поэтому ответом на вопрос "Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?" будет: качество линейной регрессии не изменится.

Затем выбрали следующий метод защиты персональной информации: умножили все имеющиеся в датасете признаки (features) на случайную обратимую матрицу, которую сгенерировали методом np.random.normal().

Замерив значения метрик R2 нашей модели LinearRegression() до и после кодирования, мы выяснили:

- до кодирования метрика R2 нашей модели составляла: 0.42494550308169177;
- после кодирования метрика R2 составила: 0.42494550308169976.

**Т.е. значения R2 до и после кодирования совпадают на 99.9999999999981%. Соответственно, мы можем полагать, что значения метрики не изменилось и цель проекта достигнута.**



<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера</b></font>


Вячеслав, у тебя старательно выполненная работа, все четко, осмысленно. Отмечаю уровень твоей программистской подготовки. 


Нет проблем с комментированием кода,




Я оставил небольшие советы и вопросики (если есть время и желание можешь воспользоваться/ответить)
    


Обязательное к исправлению:



    
- дорабатываем теоретическое доказательство, с учетом моих комментариев   


- выводим соотношение между  𝑤  и  𝑤𝑝 






Жду исправлений, для принятия проекта. Если какие то вопросы, то сразу спрашивай ) 








<div class="alert alert-warning">
<b>Комментарий ревьюера</b>



На мой взгляд проект очень необычный, выделяющийся из остальных проектов потому что касается раздела математики лежащей в основе машинного обучения  (помимо производных, матанализа и теории вероятности с матстатистикой). Если в дальнейшем ты углубишься в изучении нейростетей, то понимание линейной алгебры будет очень полезным, потому что, по сути, все нейросети - это перемножение матриц и Линейная алгебра даёт очень компактный способ записи того что происходит в нейросетях. 

Кроме того языком линейной алгебра можно объяснить тематическое моделирование и pca (через сингулярное разложение)

Мне не очень нравится эти ссылки, наверняка можно найти получше, но там вполне наглядно показывают связь между линейной алгебры и многослойным перцептроном
    
https://www.youtube.com/watch?v=kmdqBlls_HQ    
https://www.youtube.com/watch?v=AOZZeTQl4Xc


Это даже важно как "понятийный язык", вот здесь о модели word2vec (это NLP) рассказываться "языком линейной алгебры" с [22 минуты](https://www.youtube.com/watch?v=MBQdMQUZMQM&t=3340s) 
    

Возможно будет полезно:
    
    
- Теория по обратным матрицам:  [тут](https://www.berdov.com/works/matrix/obratnaya-matrica/)

    
- Если хочешь посмотреть вообще про линейную алгебру и матрицы, то мне нравится  вот это [видео](https://www.youtube.com/watch?v=RNTRYicPvWQ&list=PLVjLpKXnAGLXPaS7FRBjd5yZeXwJxZil2), рассказывается по существу и увлекательно.  


- по применению криптографии в машинном обучении на английском: [тут](https://www.cs.cmu.edu/~rjhall/JOS_revised_May_31a.pdf)




<br/>
<div class="alert alert-info">
Спасибо за ссылки! Замечания исправил. 

</div>


<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюераV2</b></font>

Спасибо за работу!    


Что осталось из красного:

    
- Неверное соотношение, как результат неверного применения матричного преобразования    

<br/>
<div class="alert alert-info">
Исправил

</div>


<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюераV3</b></font>

Спасибо за работу!    Когда правильно тогда правильно ) 

    

Красного нет, вопросов нет, значит все, пора принимать) Надеюсь мои советы и вопросики были полезны и в копилочку знаний упало что то новое, а проект стал лучше, и симпатичней.

  
Отличная работа Вячеслав. Желаю успехов в дальнейшей учебе!



</div>

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования